In [ ]:
# models.py
from django.db import models
from django.contrib.auth.models import User
from jdatetime import datetime as jdatetime
from django.utils import timezone

class Installer(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    full_name = models.CharField(max_length=100)
    phone = models.CharField(max_length=11, unique=True)
    code = models.CharField(max_length=20, unique=True, help_text="کد نصب‌کننده")
    is_active = models.BooleanField(default=True)

    def __str__(self):
        return f"{self.full_name} ({self.code})"

class Customer(models.Model):
    phone = models.CharField(max_length=11, unique=True)
    full_name = models.CharField(max_length=100, blank=True)
    created_at = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return self.phone

class Product(models.Model):
    serial_number = models.CharField(max_length=50, unique=True, verbose_name="شماره سریال")
    model_name = models.CharField(max_length=100, verbose_name="نام مدل")
    warranty_months = models.PositiveIntegerField(default=12, verbose_name="مدت گارانتی (ماه)")
    purchase_date = models.DateField(null=True, blank=True, verbose_name="تاریخ ثبت")

    def __str__(self):
        return f"{self.model_name} - {self.serial_number}"

class Installation(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    product = models.OneToOneField(Product, on_delete=models.CASCADE)
    installer = models.ForeignKey(Installer, on_delete=models.SET_NULL, null=True)
    installation_date = models.DateTimeField(default=timezone.now, verbose_name="تاریخ نصب")
    notes = models.TextField(blank=True, verbose_name="توضیحات")

    def warranty_end_date(self):
        return self.installation_date + timezone.timedelta(days=self.product.warranty_months * 30)

    def remaining_days(self):
        end = self.warranty_end_date()
        delta = end - timezone.now()
        return delta.days if delta.days > 0 else 0

    def remaining_days_jalali(self):
        end_jalali = jdatetime.fromgregorian(datetime=self.warranty_end_date())
        today_jalali = jdatetime.now()
        return (end_jalali - today_jalali).days

    def __str__(self):
        return f"{self.product} - {self.customer}"

In [ ]:
# views.py
from django.shortcuts import render, redirect, get_object_or_404
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from .models import Installation, Customer, Product, Installer
from .forms import InstallationForm, CustomerPhoneForm
from jdatetime import datetime as jdatetime

# صفحه اصلی مشتری - وارد کردن شماره موبایل
def customer_login(request):
    if request.method == 'POST':
        form = CustomerPhoneForm(request.POST)
        if form.is_valid():
            phone = form.cleaned_data['phone']
            customer, created = Customer.objects.get_or_create(phone=phone)
            request.session['customer_phone'] = phone
            return redirect('customer_panel')
    else:
        form = CustomerPhoneForm()
    return render(request, 'warranty/customer_login.html', {'form': form})

# پنل مشتری
def customer_panel(request):
    phone = request.session.get('customer_phone')
    if not phone:
        return redirect('customer_login')

    customer = get_object_or_404(Customer, phone=phone)
    installations = Installation.objects.filter(customer=customer)

    context = {
        'customer': customer,
        'installations': installations,
        'today_jalali': jdatetime.now().strftime('%Y/%m/%d'),
    }
    return render(request, 'warranty/customer_panel.html', context)

# پنل نصب‌کننده
@login_required
def installer_panel(request):
    if not hasattr(request.user, 'installer'):
        messages.error(request, "شما دسترسی ندارید.")
        return redirect('login')

    if request.method == 'POST':
        form = InstallationForm(request.POST, installer=request.user.installer)
        if form.is_valid():
            installation = form.save(commit=False)
            installation.installer = request.user.installer
            installation.save()
            messages.success(request, "نصب با موفقیت ثبت شد.")
            return redirect('installer_panel')
    else:
        form = InstallationForm(installer=request.user.installer)

    installations = Installation.objects.filter(installer=request.user.installer).order_by('-installation_date')[:50]

    return render(request, 'warranty/installer_panel.html', {
        'form': form,
        'installations': installations,
    })

In [ ]:
# forms.py
from django import forms
from .models import Installation, Customer, Product

class CustomerPhoneForm(forms.Form):
    phone = forms.CharField(
        max_length=11,
        widget=forms.TextInput(attrs={'placeholder': '09123456789', 'class': 'form-control'})
    )

class InstallationForm(forms.ModelForm):
    customer_phone = forms.CharField(max_length=11, label="شماره موبایل مشتری")
    serial_number = forms.CharField(max_length=50, label="شماره سریال محصول")

    class Meta:
        model = Installation
        fields = ['notes']
        widgets = {
            'notes': forms.Textarea(attrs={'rows': 3, 'class': 'form-control'}),
        }

    def __init__(self, *args, installer=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.installer = installer

    def clean_customer_phone(self):
        phone = self.cleaned_data['customer_phone']
        customer, _ = Customer.objects.get_or_create(phone=phone)
        return customer

    def clean_serial_number(self):
        serial = self.cleaned_data['serial_number']
        try:
            product = Product.objects.get(serial_number=serial)
            if Installation.objects.filter(product=product).exists():
                raise forms.ValidationError("این محصول قبلاً نصب شده است.")
            return product
        except Product.DoesNotExist:
            raise forms.ValidationError("شماره سریال یافت نشد.")

    def save(self, commit=True):
        installation = super().save(commit=False)
        installation.customer = self.cleaned_data['customer_phone']
        installation.product = self.cleaned_data['serial_number']
        if commit:
            installation.save()
        return installation

In [ ]:
urlpatterns = [
    path('', customer_login, name='customer_login'),
    path('panel/', customer_panel, name='customer_panel'),
    path('installer/', installer_panel, name='installer_panel'),
    path('admin/', admin.site.urls),
]

In [ ]:
INSTALLED_APPS = [
    # ...
    'django.contrib.humanize',
    'warranty',
]

TEMPLATES = [
    {
        # ...
        'OPTIONS': {
            'context_processors': [
                # ...
            ],
        },
    },
]

# برای تاریخ شمسی
from datetime import datetime
JALALI_DATE_DEFAULTS = {
    'Strftime': {
        'date': '%y/%m/%d',
        'datetime': '%y/%m/%d %H:%M',
    },
}

In [ ]:
<!-- warranty/templates/base.html -->
<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}گارانتی محصول{% endblock %}</title>
    
    <!-- Tailwind CSS CDN (v3.4+) -->
    <script src="https://cdn.tailwindcss.com"></script>
    <script>
        tailwind.config = {
            darkMode: 'class',
            theme: {
                extend: {
                    fontFamily: {
                        sans: ['Vazirmatn', 'sans-serif'],
                    }
                }
            }
        }
    </script>
    <link href="https://cdn.jsdelivr.net/gh/rastikerdar/vazirmatn@v33.003/Vazirmatn-font-face.css" rel="stylesheet" type="text/css" />
</head>
<body class="bg-gray-50 dark:bg-gray-900 text-gray-800 dark:text-gray-200 min-h-screen">
    <div class="container mx-auto px-4 py-8 max-w-5xl">
        {% block content %}{% endblock %}
    </div>
</body>
</html>

In [ ]:
<!-- warranty/templates/warranty/customer_login.html -->
{% extends 'base.html' %}
{% load static %}

{% block title %}استعلام گارانتی{% endblock %}

{% block content %}
<div class="flex flex-col items-center justify-center min-h-screen -mt-20">
    <div class="w-full max-w-md">
        <div class="text-center mb-10">
            <h1 class="text-4xl font-bold text-indigo-600 dark:text-indigo-400">استعلام گارانتی</h1>
            <p class="mt-2 text-gray-600 dark:text-gray-400">شماره موبایل خود را وارد کنید</p>
        </div>

        <form method="post" class="bg-white dark:bg-gray-800 rounded-2xl shadow-xl p-8">
            {% csrf_token %}
            <div class="mb-6">
                <label class="block text-sm font-medium mb-2">شماره موبایل</label>
                <input type="text" name="phone" maxlength="11" placeholder="09123456789"
                       class="w-full px-4 py-3 border border-gray-300 dark:border-gray-600 rounded-xl focus:ring-4 focus:ring-indigo-500 focus:border-indigo-500 dark:bg-gray-700"
                       required>
            </div>
            <button type="submit"
                    class="w-full bg-gradient-to-r from-indigo-600 to-purple-600 hover:from-indigo-700 hover:to-purple-700 text-white font-bold py-4 rounded-xl transition shadow-lg">
                مشاهده گارانتی
            </button>
        </form>
    </div>
</div>
{% endblock %}

In [ ]:
<!-- warranty/templates/warranty/customer_panel.html -->
{% extends 'base.html' %}
{% block title %}پنل مشتری{% endblock %}

{% block content %}
<div class="text-center mb-10">
    <h1 class="text-3xl font-bold text-indigo-600 dark:text-indigo-400">پنل گارانتی شما</h1>
    <p class="mt-2 text-lg">خوش آمدید، {{ customer.full_name|default:customer.phone }}</p>
    <a href="{% url 'customer_login' %}" class="text-sm text-red-500 hover:underline">خروج</a>
</div>

{% if installations %}
    <div class="grid gap-6 md:grid-cols-2 lg:grid-cols-3">
        {% for inst in installations %}
        <div class="bg-white dark:bg-gray-800 rounded-2xl shadow-lg overflow-hidden transform hover:scale-105 transition">
            <div class="bg-gradient-to-r from-indigo-500 to-purple-600 p-5 text-white">
                <h3 class="text-xl font-bold">{{ inst.product.model_name }}</h3>
                <p class="text-sm opacity-90">سریال: {{ inst.product.serial_number }}</p>
            </div>
            <div class="p-6">
                <div class="space-y-3 text-sm">
                    <div class="flex justify-between">
                        <span>تاریخ نصب:</span>
                        <span class="font-bold text-indigo-600 dark:text-indigo-400">
                            {{ inst.installation_date|date:"Y/m/d" }}
                        </span>
                    </div>
                    <div class="flex justify-between">
                        <span>پایان گارانتی:</span>
                        <span class="{% if inst.remaining_days > 30 %}text-green-600{% elif inst.remaining_days > 0 %}text-orange-600{% else %}text-red-600{% endif %} font-bold">
                            {{ inst.warranty_end_date|date:"Y/m/d" }}
                        </span>
                    </div>
                    <div class="text-center mt-4 pt-4 border-t dark:border-gray-700">
                        {% if inst.remaining_days > 0 %}
                            <span class="text-3xl font-bold text-green-600">{{ inst.remaining_days }}</span>
                            <span class="text-gray-500"> روز باقی‌مانده</span>
                        {% else %}
                            <span class="text-3xl font-bold text-red-600">منقضی شده</span>
                        {% endif %}
                    </div>
                </div>
            </div>
        </div>
        {% endfor %}
    </div>
{% else %}
    <div class="text-center py-20">
        <div class="text-6xl mb-4">📦</div>
        <p class="text-xl text-gray-500">هنوز محصولی برای شما ثبت نشده است.</p>
    </div>
{% endif %}
{% endblock %}

In [ ]:
<!-- warranty/templates/warranty/installer_panel.html -->
{% extends 'base.html' %}
{% block title %}پنل نصب‌کننده{% endblock %}

{% block content %}
<div class="flex justify-between items-center mb-8">
    <div>
        <h1 class="text-3xl font-bold">سلام، {{ request.user.installer.full_name }}</h1>
        <p class="text-gray-600 dark:text-gray-400">کد نصب‌کننده: <span class="font-bold text-indigo-600">{{ request.user.installer.code }}</span></p>
    </div>
    <a href="{% url 'logout' %}" class="bg-red-600 hover:bg-red-700 text-white px-6 py-3 rounded-xl">خروج</a>
</div>

<!-- فرم ثبت نصب جدید -->
<div class="bg-white dark:bg-gray-800 rounded-2xl shadow-xl p-8 mb-8">
    <h2 class="text-2xl font-bold mb-6 text-indigo-600">ثبت نصب جدید</h2>
    <form method="post" class="grid md:grid-cols-3 gap-6">
        {% csrf_token %}
        <div>
            <label class="block font-medium mb-2">موبایل مشتری</label>
            <input name="customer_phone" maxlength="11" placeholder="09123456789" required
                   class="w-full px-4 py-3 border rounded-xl focus:ring-4 focus:ring-indigo-500 dark:bg-gray-700">
        </div>
        <div>
            <label class="block font-medium mb-2">شماره سریال</label>
            <input name="serial_number" placeholder="ABC123456789" required
                   class="w-full px-4 py-3 border rounded-xl focus:ring-4 focus:ring-indigo-500 dark:bg-gray-700">
        </div>
        <div>
            <label class="block font-medium mb-2">توضیحات (اختیاری)</label>
            <textarea name="notes" rows="2" class="w-full px-4 py-3 border rounded-xl dark:bg-gray-700"></textarea>
        </div>
        <div class="md:col-span-3">
            <button type="submit"
                    class="w-full bg-gradient-to-r from-green-500 to-emerald-600 hover:from-green-600 hover:to-emerald-700 text-white font-bold py-4 rounded-xl">
                ثبت نصب
            </button>
        </div>
    </form>
</div>

<!-- لیست آخرین نصب‌ها -->
<h2 class="text-2xl font-bold mb-6">آخرین نصب‌ها</h2>
<div class="grid gap-4">
    {% for inst in installations %}
    <div class="bg-white dark:bg-gray-800 rounded-xl shadow p-6 flex justify-between items-center">
        <div>
            <p class="font-bold">{{ inst.product.model_name }} - {{ inst.product.serial_number }}</p>
            <p class="text-sm text-gray-500">مشتری: {{ inst.customer.phone }} | تاریخ: {{ inst.installation_date|date:"Y/m/d H:i" }}</p>
        </div>
        <span class="bg-green-100 text-green-800 dark:bg-green-900 dark:text-green-200 px-4 py-2 rounded-full text-sm font-bold">
            ثبت شده
        </span>
    </div>
    {% empty %}
    <p class="text-center text-gray-500 py-10">هنوز نصبی ثبت نکرده‌اید.</p>
    {% endfor %}
</div>
{% endblock %}

In [ ]:
<!-- Toast Notification - خیلی شیک و حرفه‌ای -->
<div x-data="{ showToast: false, toastMessage: '', toastType: 'success' }"
     x-init="() => {
         window.addEventListener('show-toast', event => {
             toastMessage = event.detail.message;
             toastType = event.detail.type || 'success';
             showToast = true;
             setTimeout(() => showToast = false, 5000);
         });
     }"
     x-show="showToast"
     x-transition:enter="transition ease-out duration-300"
     x-transition:enter-start="transform translate-x-full opacity-0"
     x-transition:enter-end="transform translate-x-0 opacity-100"
     x-transition:leave="transition ease-in duration-300"
     x-transition:leave-start="transform translate-x-0 opacity-100"
     x-transition:leave-end="transform translate-x-full opacity-0"
     class="fixed top-4 left-4 z-50 max-w-sm"
     dir="rtl">
     
    <div :class="{
        'bg-green-600': toastType === 'success',
        'bg-red-600': toastType === 'error',
        'bg-amber-500': toastType === 'warning',
        'bg-blue-600': toastType === 'info'
    }" class="text-white px-6 py-4 rounded-2xl shadow-2xl flex items-center gap-3">
        <span x-text="toastMessage"></span>
        <button @click="showToast = false" class="text-white hover:text-gray-200">✕</button>
    </div>
</div>

<!-- Alpine.js CDN -->
<script src="https://cdn.jsdelivr.net/npm/alpinejs@3.x.x/dist/cdn.min.js" defer></script>

In [ ]:
<script>
    function showToast(message, type = 'success') {
        window.dispatchEvent(new CustomEvent('show-toast', {
            detail: { message, type }  // success, error, warning, info
        }));
    }
</script>

In [ ]:
# به جای messages.success(request, "ثبت شد")
# این رو بنویس:

return render(request, 'warranty/installer_panel.html', {
    'form': form,
    'installations': installations,
}) + f"""
<script>
    showToast('نصب با موفقیت ثبت شد ✓', 'success');
</script>
"""

# یا برای خطاها:
return render(request, 'warranty/installer_panel.html', {
    'form': form,
    'installations': installations,
}) + f"""
<script>
    showToast('شماره سریال اشتباه است یا قبلاً استفاده شده!', 'error');
</script>
"""

In [ ]:
@login_required
def installer_panel(request):
    if not hasattr(request.user, 'installer'):
        script = "<script>showToast('شما دسترسی ندارید!', 'error'); setTimeout(() => location.href='/', 2000);</script>"
        return HttpResponse(script)

    if request.method == 'POST':
        form = InstallationForm(request.POST, installer=request.user.installer)
        if form.is_valid():
            form.save()
            script = "<script>showToast('نصب با موفقیت ثبت شد ✅', 'success');</script>"
        else:
            errors = ' | '.join([f"{k}: {v[0]}" for k, v in form.errors.items()])
            script = f"<script>showToast('خطا: {errors}', 'error');</script>"
    else:
        form = InstallationForm(installer=request.user.installer)
        script = ""

    installations = Installation.objects.filter(installer=request.user.installer)[:50]

    response = render(request, 'warranty/installer_panel.html', {
        'form': form,
        'installations': installations,
    })
    response.content += script.encode()
    return response

In [ ]:
def customer_login(request):
    if request.method == 'POST':
        form = CustomerPhoneForm(request.POST)
        if form.is_valid():
            phone = form.cleaned_data['phone']
            customer, _ = Customer.objects.get_or_create(phone=phone)
            request.session['customer_phone'] = phone
            script = "<script>showToast('خوش آمدید 👋', 'success');</script>"
            return redirect('customer_panel') 
        else:
            script = "<script>showToast('شماره موبایل معتبر نیست!', 'error');</script>"
    else:
        form = CustomerPhoneForm()
        script = ""

    response = render(request, 'warranty/customer_login.html', {'form': form})
    if script:
        response.content += script.encode()
    return response